# STEP01 Create Training Corpus Tutorial

This pipeline turns input text or text collections (See Figure 1) into a form that can be used to train BERT models using the next pipelines in this project. The result of this pipeline is a .tsv file with one column called "text". Each row in that file contains a document or consecutive sentences. The sentences are separated by "\n".

![step01](images/step01.png)
**Figure 1.** Pipeline visualized

In [1]:
import csv
from estnltk.corpus_processing.parse_koondkorpus import parse_tei_corpora
from estnltk import Text

def showFirstN(n, path):
    with open(path, newline='', encoding='utf-8') as file:
        tsv_reader = csv.reader(file, delimiter="\t")
        next(tsv_reader)
        for i, row in enumerate(tsv_reader):
            if (i == n):
                break
            print(row)

## ESTNLTK Corpus processing

These pipelines do not clean/replace tokens in texts unless specified. Here is a basic example:

In [2]:
from pipelines.step01_create_training_corpus.estNLTK_corpus_processing import clean_and_extract_sentences_corpus


corpus_path = "data/Horisont/Hori/horisont"
result_path = "data/corp_res_clean.tsv"

# showing the first entry in the corpus
for text_obj in parse_tei_corpora(corpus_path, target=['artikkel']):
    display(text_obj)
    break

# processing and showing the first entry
clean_and_extract_sentences_corpus(corpus_path, result_path)
showFirstN(1, result_path)
        

Text(text='MARIA TEIVERLAUR\n\nMina olen veel noor …\n\nMARIA TEIVERLAUR ( 1955 ) on meditsiinipsühholoog , Jämejala Psühhiaatriahaigla psühholoogia kabineti juhataja .\nViljandi Kultuurikolledzi õppejõud .\nPedagoogikakandidaat aastast 1992.\n\nTeie käest küsitakse , kui vana Te olete .\nTundub , et sellele pärimisele on lihtne vastata .\nAga võta näpust … nüüdisaja teadus eristab hoopis mitut erinevat vanust , sealhulgas näiteks bioloogilist , sotsiaalset ja psühholoogilist .\n\nInimese subjektiivne suhtumine oma vanusesse on küllaltki erinev .\nMõnigi eakas inimene võib tunda end nooremana , teine vanemana kui elatud aastad näitavad .\nElus juhtub sageli , et inimene peab end nooremaks , kui ta tegelikult ( n.-ö. kronoloogiliselt ) o n.\n\nMõned aastad tagasi korraldati katse , milles osales 83 kõrgharidusega meest ja naist vanuses 21-44 aastat .\nNad pidid ette kujutama , et ei tea oma tõelist vanust , ning andma subjektiivse hinnangu selle kohta .\nTulemused näitasid , et ligi veerandil tegel ik ja subjektiivne vanus kattusid kas täiesti või piirdus erinevus aastaga .\nÜle poole katsealustest pidas end nooremaks ja umbes viiendik tundis end vanemana , kui nad tegelikult olid .\n\nNooruse saladused\n\nSubjektiivne arvamus oma vanuse kohta on üsna oluline , sest enesetunnetusest selle kõige laiemas tähenduses sõltub ju inimese meeleolu ja käitumine .\nMiks inimesed oma vanust erinevalt tunnetavad , on püütud seletada mitut moodi .\nTeadlaste seas ühe levi nuima arvamuse järgi on igal isiksusel nn. aastate psühhofüsioloogilise aktiivsuse tsüklite “ loendaja ” , mille “ näitude ” alusel hinnataksegi oma psühholoogilist vanust .\nTeine seletus opereerib saavutustega , eneseteostusega , mis on teatud eas inimestele ena masti omane .\nRõhutatakse , et kui inimese saavutused ületavad sotsiaalsed ootused , hakkab ta end tundma vanemana , kui ta tegelikult on .\nJuhul , kui saavutused on väiksemad kui teatud vanuses temalt loodetakse , tunnetab isik end nooremana .\nKatsed näitavad , e t näiteks vallalised tunnetavad end oluliselt nooremana kui abieluinimesed .\nInimesed , kes tunnetavad end tegelikust tunduvalt nooremana , on sageli vähem iseseisvad ja emotsionaalselt ebastabiilsemad .\n\nSeega võib psühholoogilist vanust defineerida ka kui isiksuse psühholoogilise aja realiseerumise astet .\nVõi teisiti - psühholoogilise aja realiseerumist ehk täitumist teadvustab iga inimene oma “ hingelise ” vanusena , mida nimetataksegi isiksuse psühhol oogiliseks vanuseks .\nErinevalt kronoloogilisest ja bioloogilisest vanusest on psühholoogiline vanus pöörduv , s.t. et inimene võib muutuda järjest nooremaks .\nSee toimub enamasti n.-ö. psühholoogilise tuleviku suurenemise arvel : kui inimene loodab kaua elad a , siis ta vananeb “ aeglasemalt ” .\n\nMõtted , mis teevad vanemaks või nooremaks\n\nNagu teada , elavad paljud vanad inimesed oma mõtetega rohkem minevikus .\nSellega püüavad nad kompenseerida oma bioloogilist ja sotsiaalset vananemist .\nElades mineviku kujutlustes oma ilusate mälestuste keskel tunneb vana inimene end emotsionaalselt tea taval määral nooremana .\nTäpsustuseks peab veel lisama , et see , kes elab mõtetega rohkem tulevikus , kui olevikus , tunnetab end vanemana , kui inimene , kes “ elab siin ja praegu ” .\n\nPraktika näitab veel ka , et lootuse kaotanud inimese ja optimistlikult meelestatud inimese psühholoogilise mineviku ja tuleviku suhe ning psühholoogilise aja täitumus erinevad oluliselt .\nPsühholoogiline vanus on seda suurem , mida vähem inimene loodab veel elada ja mida enam on realiseerunud tema psühholoogiline aeg .\n\nMärkimist väärib fakt , et erinevates eluvaldkondades ( tööl , kodus , sõprade keskel jne. ) hindab inimene oma psühholoogilise aja täitumust erinevalt , seega on vastavalt tegutsemise sfäärile erinev ka tema psühholoogiline vanus .\nIgal juhul on lootus ja h uvi oma tuleviku vastu isiksuse harmooniliseks psühholoogiliseks arenguks väga oluline .\nInimene , kellele tulevik ei tõota meeldivat te

['MARIA TEIVERLAURMina olen veel noor …MARIA TEIVERLAUR ( 1955 ) on meditsiinipsühholoog , Jämejala Psühhiaatriahaigla psühholoogia kabineti juhataja .\nViljandi Kultuurikolledzi õppejõud .\nPedagoogikakandidaat aastast 1992.\nTeie käest küsitakse , kui vana Te olete .\nTundub , et sellele pärimisele on lihtne vastata .\nAga võta näpust … nüüdisaja teadus eristab hoopis mitut erinevat vanust , sealhulgas näiteks bioloogilist , sotsiaalset ja psühholoogilist .\nInimese subjektiivne suhtumine oma vanusesse on küllaltki erinev .\nMõnigi eakas inimene võib tunda end nooremana , teine vanemana kui elatud aastad näitavad .\nElus juhtub sageli , et inimene peab end nooremaks , kui ta tegelikult ( n.-ö. kronoloogiliselt ) o n .\nMõned aastad tagasi korraldati katse , milles osales 83 kõrgharidusega meest ja naist vanuses 21 - 44 aastat .\nNad pidid ette kujutama , et ei tea oma tõelist vanust , ning andma subjektiivse hinnangu selle kohta .\nTulemused näitasid , et ligi veerandil tegel ik ja s

### ESTNLTK Corpus processing + clean

You have to specify the *clean* argument to clean or further process these texts. If specified, this argument must be a function that takes 1 argument (an EstNLTK Text object) and also returns an EstNLTK Text object or a list of EstNLTK Text objects.

In [3]:
# lets make a simple clean function that removes every other token.
def simpleClean(text_obj):
    text_obj.tag_layer(['tokens'])
    res = []
    for i, token in enumerate(text_obj.tokens):
        if (i % 2 == 0):
            res.append(token.text)
    return Text(" ".join(res))
    
# testing
simpleClean(Text("Täna on äärmiselt kena ilm."))

text
Täna äärmiselt ilm


In [4]:
# processing and showing the first entry
clean_and_extract_sentences_corpus(corpus_path, result_path, clean=simpleClean)
showFirstN(1, result_path)

['MARIA Mina veel …TEIVERLAUR 1955 on , Psühhiaatriahaigla kabineti .Kultuurikolledzi .aastast .\nkäest , vana olete Tundub et pärimisele lihtne .\nvõta … teadus hoopis erinevat , näiteks , ja .\nsubjektiivne oma on erinev Mõnigi inimene tunda nooremana teine kui aastad .\njuhtub , inimene end , ta ( .\nö kronoloogiliselt o .\naastad korraldati , osales kõrgharidusega ja vanuses - aastat Nad ette , ei oma vanust ning subjektiivse selle .\nnäitasid et veerandil ik subjektiivne kattusid täiesti piirdus aastaga Üle katsealustest end ja viiendik end , nad olid Nooruse Subjektiivne oma kohta üsna , enesetunnetusest kõige tähenduses ju meeleolu käitumine Miks oma erinevalt , püütud mitut .\nseas levi arvamuse on isiksusel .\npsühhofüsioloogilise tsüklite loendaja , “ ” hinnataksegi psühholoogilist .\nseletus saavutustega eneseteostusega mis teatud inimestele masti .\n, kui saavutused sotsiaalsed , ta tundma , ta on Juhul kui on kui vanuses loodetakse tunnetab end .\nnäitavad e näiteks tunnet

### ESTNLTK pre-made cleaning functions

There are two cleaning functions already made in this project. Both are meant for cleaning clinical notes. These are called:

 * **clean_med** - removes tables/lists, replaces anonymised tags with "ta"/"tegema" (in the correct form) /"\<ADJ>"/"\<NAME>"/"\<ADV>"/"\<INJ>", replaces dates with "\<DATE>", integers with "\<INT>" and floats with "\<FLOAT>"
 * **clean_med_r_events** - does the same as **clean_med**, but also removes the event headers and splits the clinical note at event header locations into individual tokens.

In [5]:
from pipelines.step01_create_training_corpus.textprocessing.text_cleaning import clean_med, clean_med_r_events

clean_and_extract_sentences_corpus(corpus_path, result_path, clean=clean_med)
showFirstN(1, result_path)

['MARIA TEIVERLAURMina olen veel noor …MARIA TEIVERLAUR ( <DATE> ) on meditsiinipsühholoog , Jämejala Psühhiaatriahaigla psühholoogia kabineti juhataja .\nViljandi Kultuurikolledzi õppejõud .\nPedagoogikakandidaat aastast <DATE> .\nTeie käest küsitakse , kui vana Te olete .\nTundub , et sellele pärimisele on lihtne vastata .\nAga võta näpust … nüüdisaja teadus eristab hoopis mitut erinevat vanust , sealhulgas näiteks bioloogilist , sotsiaalset ja psühholoogilist .\nInimese subjektiivne suhtumine oma vanusesse on küllaltki erinev .\nMõnigi eakas inimene võib tunda end nooremana , teine vanemana kui elatud aastad näitavad .\nElus juhtub sageli , et inimene peab end nooremaks , kui ta tegelikult ( n.-ö. kronoloogiliselt ) o n .\nMõned aastad tagasi korraldati katse , milles osales <INT> kõrgharidusega meest ja naist vanuses <INT> - <INT> aastat .\nNad pidid ette kujutama , et ei tea oma tõelist vanust , ning andma subjektiivse hinnangu selle kohta .\nTulemused näitasid , et ligi veerandil

## Other pipelines in step01

Other pipelines behave similarly, thus wont be shown in this tutorial, but they are brought out here:

 * **clean_and_extract_sentences_db** - Takes texts ('raw_text' column) from source table.schema, cleans then separates sentences and puts the result into target schema.table (text column)
        param: db_config: parameters used to make a connection to the database
                for example:
                "postgresql://{username}:{password}@{host}:{port}/{database}".format(
                username="username",
                password="pass",
                host="ip",
                port=5432,
                database="dbname")
        :param source_schema: The schema, that contains the source_table
        :param source_table: The table that contains the texts which will be processed ('raw_text' column)
        :param target_schema: The schema, that contains the target_table
        :param target_table: The name of the resulting table. Must have column: 'text'
        :param clean: Function, that cleans takes an EstNLTK object as an argument and cleans it and returns it
            
 * **clean_and_extract_sentences_tsv** - A pipeline that converts and cleans text from a source tsv file into usable form for training a BERT model.
        :param in_tsv_path: path to the source .tsv file
        :param out_path: path to the output .txt file
        :param text_col_i: index of the column, that contains text to be cleaned and extracted
        :param clean: Function, that cleans takes an EstNLTK object as an argument and cleans it
        
 * **clean_and_extract_parallel_tsv** - A parallelized pipeline that converts and cleans text from a source tsv file into usable form for training a BERT model.
        :param max_processes: (default = 3) (min = 3), The number of processes you want to create and use. Note that you don't need as many if you use a cheap clean function (if at all) than an expensive one.
        For other params see clean_and_extract_sentences_tsv
 
 
 * **clean_and_extract_parallel_corpus** - A parallelized pipeline that converts and cleans text from a source tsv file into usable form for training a BERT model.
        :param max_processes: (default = 3) (min = 3), The number of processes you want to create and use. Note that you don't need as many if you use a cheap clean function (if at all) than an expensive one.
        For other params see clean_and_extract_parallel_corpus
 